In [ ]:
import pandas as pd
import pandas_ta as ta
import plotly.graph_objects as go

# Load data
data = pd.read_csv("RNDRUSDT_1.csv", parse_dates=['date'], index_col='date')

# Calculate VWAP
data.ta.vwap(append=True)

data

,open,high,low,close,volume,number_of_trades,VWAP_D
date,,,,,,,
2021-11-27,6.100,7.945,2.500,7.470,5385361.76,72851,5.971667
2021-11-28,7.470,7.827,6.083,6.948,2991578.89,33016,6.952667
2021-11-29,6.968,7.220,6.333,6.788,1362160.41,20534,6.780333
2021-11-30,6.795,6.824,6.138,6.268,1126567.79,16126,6.410000
2021-12-01,6.276,6.360,5.145,5.401,3006782.57,41587,5.635333
...,...,...,...,...,...,...,...
2024-05-24,10.234,10.342,9.750,10.110,5285923.03,196654,10.067333
2024-05-25,10.110,10.449,10.029,10.077,3818263.45,125499,10.185000
2024-05-26,10.079,10.179,9.864,9.988,3195301.24,109498,10.010333


In [ ]:
# Calculate Bollinger Bands using pandas_ta
data.ta.bbands(length=10, std=1.5, append=True)

# Add the upper and lower bands to the DataFrame
data['Upper Band'] = data['BBU_10_1.5']
data['Lower Band'] = data['BBL_10_1.5']

def calculate_sma(data, length: int):
    return ta.sma(data['close'], length)

# Calculate the moving average
data['SMA'] = calculate_sma(data, 20)
data.dropna(inplace=True)

data

,open,high,low,close,volume,number_of_trades,VWAP_D,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA
date,,,,,,,,,,,,,,,
2022-01-04,5.264,5.500,4.764,4.813,3928565.29,62107,5.025667,4.441881,4.8765,5.311119,17.825050,0.426948,5.311119,4.441881,4.97220
2022-01-05,4.814,5.122,4.030,4.278,3805854.49,50055,4.476667,4.346060,4.7621,5.178140,17.472977,-0.081795,5.178140,4.346060,4.97165
2022-01-06,4.278,4.380,4.063,4.172,3791357.58,49626,4.205000,4.222404,4.6637,5.104996,18.924732,-0.057109,5.104996,4.222404,4.97205
2022-01-07,4.175,4.180,3.645,3.686,3518981.52,44164,3.837000,3.939141,4.5416,5.144059,26.530685,-0.210090,5.144059,3.939141,4.93135
2022-01-08,3.693,3.850,3.407,3.580,2230354.95,27393,3.612333,3.699075,4.4374,5.175725,33.277383,-0.080638,5.175725,3.699075,4.87595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-24,10.234,10.342,9.750,10.110,5285923.03,196654,10.067333,9.851331,10.4206,10.989869,10.925831,0.227194,10.989869,9.851331,10.45310
2024-05-25,10.110,10.449,10.029,10.077,3818263.45,125499,10.185000,9.787512,10.3645,10.941488,11.133927,0.250861,10.941488,9.787512,10.46350
2024-05-26,10.079,10.179,9.864,9.988,3195301.24,109498,10.010333,9.774412,10.3589,10.943388,11.284758,0.182714,10.943388,9.774412,10.46555


In [ ]:
def check_candles(data, backcandles, ma_column):
    categories = [0 for _ in range(backcandles)]
    for i in range(backcandles, len(data)):
        if all(data['close'][i-backcandles:i] > data[ma_column][i-backcandles:i]):
            categories.append(2)  # Uptrend
        elif all(data['close'][i-backcandles:i] < data[ma_column][i-backcandles:i]):
            categories.append(1)  # Downtrend
        else:
            categories.append(0)  # No trend
    return categories

# Apply the function to the DataFrame
data['Trend'] = check_candles(data, 7, 'SMA')

data

,open,high,low,close,volume,number_of_trades,VWAP_D,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend
date,,,,,,,,,,,,,,,,
2022-01-04,5.264,5.500,4.764,4.813,3928565.29,62107,5.025667,4.441881,4.8765,5.311119,17.825050,0.426948,5.311119,4.441881,4.97220,0
2022-01-05,4.814,5.122,4.030,4.278,3805854.49,50055,4.476667,4.346060,4.7621,5.178140,17.472977,-0.081795,5.178140,4.346060,4.97165,0
2022-01-06,4.278,4.380,4.063,4.172,3791357.58,49626,4.205000,4.222404,4.6637,5.104996,18.924732,-0.057109,5.104996,4.222404,4.97205,0
2022-01-07,4.175,4.180,3.645,3.686,3518981.52,44164,3.837000,3.939141,4.5416,5.144059,26.530685,-0.210090,5.144059,3.939141,4.93135,0
2022-01-08,3.693,3.850,3.407,3.580,2230354.95,27393,3.612333,3.699075,4.4374,5.175725,33.277383,-0.080638,5.175725,3.699075,4.87595,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-05-24,10.234,10.342,9.750,10.110,5285923.03,196654,10.067333,9.851331,10.4206,10.989869,10.925831,0.227194,10.989869,9.851331,10.45310,0
2024-05-25,10.110,10.449,10.029,10.077,3818263.45,125499,10.185000,9.787512,10.3645,10.941488,11.133927,0.250861,10.941488,9.787512,10.46350,0
2024-05-26,10.079,10.179,9.864,9.988,3195301.24,109498,10.010333,9.774412,10.3589,10.943388,11.284758,0.182714,10.943388,9.774412,10.46555,0


In [ ]:
## Entry Based on Bollinger Bands - Candles Crossing

In [ ]:
# Check conditions and assign entry values
data['entry'] = 0

# Condition for entry category 2 (buy entry)
buy_entry_condition = (data['Trend'] == 2) & ((data['open'] < data['Lower Band']) & (data['close'] > data['Lower Band']))
data.loc[buy_entry_condition, 'entry'] = 2

# Condition for entry category 1 (sell entry)
sell_entry_condition = (data['Trend'] == 1) & ((data['open'] > data['Upper Band']) & (data['close'] < data['Upper Band']))
data.loc[sell_entry_condition, 'entry'] = 1

data[data['entry']!=0]

,open,high,low,close,volume,number_of_trades,VWAP_D,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry
date,,,,,,,,,,,,,,,,,
2022-04-20,2.144,2.207,2.024,2.053,2405742.23,18030,2.094667,1.983251,2.0508,2.118349,6.587606,0.516284,2.118349,1.983251,2.38835,1,1
2022-09-27,0.486,0.502,0.470,0.477,2336607.26,6740,0.483000,0.442692,0.4640,0.485308,9.184676,0.805043,0.485308,0.442692,0.49705,1,1
2023-02-10,1.375,1.525,1.318,1.483,40702255.62,177678,1.442000,1.393864,1.6981,2.002336,35.832482,0.146491,2.002336,1.393864,1.35000,2,2
2023-08-26,1.478,1.502,1.410,1.421,2776607.08,14138,1.444333,1.388292,1.4304,1.472508,5.887594,0.388382,1.472508,1.388292,1.53980,1,1


In [ ]:
import plotly.graph_objects as go

dfpl = data[:]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'])])

# Add the moving averages to the plot
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['SMA'], mode='lines', name='SMA', line=dict(color='red')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['Lower Band'], mode='lines', name='Lower Band', line=dict(color='blue')))
fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['Upper Band'], mode='lines', name='Upper Band', line=dict(color='blue')))

fig.update_layout(xaxis_rangeslider_visible=False)
fig.show()

In [ ]:
## 2 - Entry based on RSI and Bollinger Bands

In [ ]:
def add_rsi_column(data):
    # Calculate RSI with a period of 14
    data['RSI'] = ta.rsi(data['close'])
    return data

data = add_rsi_column(data)

In [ ]:
def rsi_signal(data):
    data['RSI Signal'] = 0  # Initialize the signal column with 0

    # Set the signal category to 2 when the price is below the lower Bollinger Band and RSI is below 30
    data.loc[(data['close'] < data['Lower Band']) & (data['RSI'] < 55), 'RSI Signal'] = 2

    # Set the signal category to 1 when the price is above the upper Bollinger Band and RSI is above 70
    data.loc[(data['close'] > data['Upper Band']) & (data['RSI'] > 45), 'RSI Signal'] = 1

    return data

In [ ]:
data = rsi_signal(data)

data[data["RSI Signal"]!=0]

,open,high,low,close,volume,number_of_trades,VWAP_D,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,RSI,RSI Signal
date,,,,,,,,,,,,,,,,,,,
2022-01-18,3.197,3.319,2.880,3.000,2447852.63,26407,3.066333,3.136695,3.5220,3.907305,21.879898,-0.177385,3.907305,3.136695,3.97970,1,0,23.941586,2
2022-01-19,3.000,3.187,2.758,2.965,2757651.67,31158,2.970000,3.005314,3.4422,3.879086,25.384134,-0.046137,3.879086,3.005314,3.89745,1,0,23.515635,2
2022-01-20,2.964,3.372,2.820,2.829,2467611.47,27615,3.007000,2.860762,3.3752,3.889638,30.483423,-0.030870,3.889638,2.860762,3.80885,1,0,21.886215,2
2022-01-21,2.827,2.901,2.262,2.415,5283958.03,45158,2.526000,2.602438,3.2692,3.935962,40.790509,-0.140559,3.935962,2.602438,3.68960,1,0,17.834922,2
2022-01-22,2.413,2.503,1.962,2.145,3383954.01,30081,2.203333,2.349822,3.0865,3.823178,47.735504,-0.139017,3.823178,2.349822,3.56825,1,0,15.783001,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-30,7.886,8.000,6.987,7.339,7119578.24,232164,7.442000,7.510966,8.3841,9.257234,20.828327,-0.098476,9.257234,7.510966,8.32525,0,0,36.512873,2
2024-05-04,8.241,9.100,8.228,8.958,6003436.43,194298,8.762000,7.411065,8.0698,8.728535,16.325922,1.174171,8.728535,7.411065,8.27610,1,0,54.192347,1
2024-05-05,8.958,10.000,8.770,9.869,11339121.31,359250,9.546333,7.165603,8.1949,9.224197,25.120439,1.313225,9.224197,7.165603,8.35555,0,0,61.240100,1


In [ ]:
# Condition for entry category 2 (buy entry)
buy_entry_condition = (data['Trend'] == 2) & (data['RSI Signal'] == 2) & (data['low'] < data['Lower Band'])
data.loc[buy_entry_condition, 'entry'] = 2

# Condition for entry category 1 (sell entry)
sell_entry_condition = (data['Trend'] == 1) & (data['RSI Signal'] == 1) & (data['high'] > data['Upper Band'])
data.loc[sell_entry_condition, 'entry'] = 1

data[data['entry']!=0]

,open,high,low,close,volume,number_of_trades,VWAP_D,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,RSI,RSI Signal
date,,,,,,,,,,,,,,,,,,,
2022-01-29,2.837,3.247,2.835,3.179,4572167.03,46466,3.087000,2.056366,2.5218,2.987234,36.912857,1.206007,2.987234,2.056366,2.98200,1,1,49.203448,1
2022-02-10,3.327,3.480,3.020,3.075,4585635.63,46795,3.191667,3.159825,3.4702,3.780575,17.888029,-0.136649,3.780575,3.159825,3.06230,2,2,44.410911,2
2022-02-11,3.077,3.159,2.581,2.677,4034861.06,40119,2.805667,2.921076,3.3639,3.806724,26.327982,-0.275591,3.806724,2.921076,3.08890,2,2,38.057872,2
2022-02-28,2.384,2.687,2.357,2.674,3062683.64,20990,2.572667,2.300718,2.4693,2.637882,13.654206,1.107124,2.637882,2.300718,2.69020,1,1,47.501784,1
2022-03-01,2.674,3.125,2.531,2.979,5403659.82,48522,2.878333,2.224438,2.5058,2.787162,22.456893,1.340908,2.787162,2.224438,2.67270,1,1,53.960338,1
2022-04-05,2.904,2.979,2.777,2.806,1587329.63,16911,2.854000,2.813327,2.9373,3.061273,8.441294,-0.029550,3.061273,2.813327,2.81180,2,2,50.363184,2
2022-04-20,2.144,2.207,2.024,2.053,2405742.23,18030,2.094667,1.983251,2.0508,2.118349,6.587606,0.516284,2.118349,1.983251,2.38835,1,1,36.325021,0
2022-06-23,0.493,0.620,0.492,0.555,27518718.64,46656,0.555667,0.312967,0.4153,0.517633,49.281318,1.182578,0.517633,0.312967,0.52405,1,1,46.807661,1
2022-09-08,0.508,0.697,0.490,0.551,13063378.82,36461,0.579333,0.484296,0.5135,0.542704,11.374415,1.142040,0.542704,0.484296,0.53685,1,1,49.101314,1


In [ ]:
## 3 - Entry based on a rejection candle next to Bollinger Bands

In [ ]:
def identify_shooting_star(data):
    # Create a new column for shooting star
    data['shooting_star'] = data.apply(lambda row: 2 if (
        ( (min(row['open'], row['close']) - row['low']) > (1.5 * abs(row['close'] - row['open']))) and 
        (row['high'] - max(row['close'], row['open'])) < (0.8 * abs(row['close'] - row['open'])) and 
        (abs(row['open'] - row['close']) > row['open'] * 0.01)
    ) else 1 if (
        (row['high'] - max(row['open'], row['close'])) > (1.5 * abs(row['open'] - row['close'])) and 
        (min(row['close'], row['open']) - row['low']) < (0.8 * abs(row['open'] - row['close'])) and 
        (abs(row['open'] - row['close']) > row['open'] * 0.01)
    ) else 0, axis=1)

    return data

data = identify_shooting_star(data)

data[data['shooting_star']!=0]

,open,high,low,close,volume,number_of_trades,VWAP_D,BBL_10_1.5,BBM_10_1.5,BBU_10_1.5,BBB_10_1.5,BBP_10_1.5,Upper Band,Lower Band,SMA,Trend,entry,RSI,RSI Signal,shooting_star
date,,,,,,,,,,,,,,,,,,,,
2022-01-20,2.964,3.372,2.820,2.829,2467611.47,27615,3.007000,2.860762,3.3752,3.889638,30.483423,-0.030870,3.889638,2.860762,3.80885,1,0,21.886215,2,1
2022-02-13,2.694,2.976,2.486,2.566,4230670.72,38892,2.676000,2.598047,3.2272,3.856353,38.990638,-0.025468,3.856353,2.598047,3.12565,0,0,36.526607,2,1
2022-02-18,2.791,3.009,2.671,2.696,2882244.84,32666,2.792000,2.517830,2.9111,3.304370,27.018640,0.226524,3.304370,2.517830,3.19895,0,0,41.823791,0,1
2022-02-21,2.465,2.880,2.301,2.361,4991440.66,56282,2.514000,2.335241,2.7468,3.158359,29.966434,0.031294,3.158359,2.335241,3.05535,0,0,36.707987,0,1
2022-02-22,2.363,2.504,2.215,2.460,3669058.38,34023,2.393000,2.292090,2.7234,3.154710,31.674414,0.194652,3.154710,2.292090,3.01535,0,0,39.240174,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-04-12,9.029,9.237,6.628,8.185,14095389.86,652153,8.016667,8.588394,9.3661,10.143806,16.606822,-0.259349,10.143806,8.588394,10.04685,1,0,36.116449,2,2
2024-04-13,8.184,8.356,6.268,7.480,17080078.18,830846,7.368000,8.026968,9.1785,10.330032,25.091947,-0.237496,10.330032,8.026968,9.87450,1,0,32.172846,2,2
2024-04-23,9.275,9.747,8.870,8.980,6623991.04,242469,9.199000,7.688881,8.5316,9.374319,19.755251,0.766043,9.374319,7.688881,8.85505,0,0,49.121739,0,1
